In [1]:
!pip install faiss-cpu
!pip install pdfplumber


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.6 MB/s eta 0:00:00


In [3]:
import os
import faiss
import numpy as np
import torch
from transformers import pipeline, AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import pdfplumber


In [4]:
# Step 1: Load and Preprocess the Book
def load_book(filepath):
    """Reads a text or PDF file and returns its content as a string."""
    if filepath.endswith(".txt"):
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()
    elif filepath.endswith(".pdf"):
        text = ""
        with pdfplumber.open(filepath) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text
    else:
        raise ValueError("Unsupported file format. Use .txt or .pdf")

In [5]:

# Step 2: Split Text into Chunks
def split_text(text, chunk_size=500):
    """Splits text into smaller chunks for retrieval."""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [6]:

# Step 3: Convert Text Chunks into Embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
def create_embeddings(chunks):
    return embedding_model.encode(chunks, convert_to_numpy=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Step 4: Store Embeddings in FAISS Index
def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


In [8]:
# Step 5: Retrieve Relevant Chunks
def retrieve_passage(question, chunks, index, top_k=3):
    query_vector = embedding_model.encode([question])
    distances, indices = index.search(query_vector, top_k)
    return " ".join([chunks[i] for i in indices[0]])


In [9]:
# Step 6: Load a Question Answering Model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_question(question, context):
    return qa_pipeline(question=question, context=context)["answer"]


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [21]:
# Step 7: Run the Model
if __name__ == "__main__":
    book_text = load_book(r"/content/Dr._Kalam-profile.pdf")
    chunks = split_text(book_text)
    embeddings = create_embeddings(chunks)
    faiss_index = build_faiss_index(embeddings)

    question = "who is kalam.?"
    retrieved_context = retrieve_passage(question, chunks, faiss_index)
    answer = answer_question(question, retrieved_context)

    print(f"Q: {question}\nA: {answer}")

Q: who is kalam.?
A: one of the most distinguished scientists of India


In [22]:
import faiss
import pickle

# Save the FAISS index
faiss.write_index(faiss_index, "faiss_index.bin")

# Save the chunks (as you'll need them later)
with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)


In [23]:
embedding_model.save("embedding_model")
